In [1]:
import pandas as pd
import os
import csv
from pathlib import Path
import hvplot.pandas
import requests
import numpy
from scipy.stats import linregress 


In [2]:
# Input cvs raw data to data frames
delays_df = pd.read_csv("Raw_data/Airline_Delay_Cause_cleaned.csv")
delays_df.head()

,ID,year,month,carrier,carrier_name,airport,airport_name,arr_flights,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted
0,1,2024,5,UA,United Air Lines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",637.0,52.27,9.44,62.80,0.0,68.49,9.0,1.0
1,2,2024,5,UA,United Air Lines Network,AUS,"Austin, TX: Austin - Bergstrom International",780.0,62.17,10.39,67.55,0.0,76.88,13.0,4.0
2,3,2024,5,UA,United Air Lines Network,BNA,"Nashville, TN: Nashville International",603.0,45.67,6.26,45.91,0.0,58.15,3.0,3.0
3,4,2024,5,UA,United Air Lines Network,BOS,"Boston, MA: Logan International",1092.0,75.30,22.35,124.17,0.0,87.19,14.0,4.0
4,5,2024,5,UA,United Air Lines Network,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",326.0,29.85,2.95,15.36,0.0,30.83,3.0,0.0
